# Group assignment DL - Accent classification

## Data pre-processing

In [14]:
# %pip install torch
# %pip install -r requirements.txt 
import os
import pandas as pd
import torch
# import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F


In [15]:
%pip install torchaudio

DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch, torchaudio
print(torch.__version__, torchaudio.__version__)

2.6.0 2.6.0


## FIX FIX FIXXX

In [17]:
%pip uninstall -y torch torchvision torchaudio torch_directml

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
Found existing installation: torchvision 0.22.0
Uninstalling torchvision-0.22.0:
  Successfully uninstalled torchvision-0.22.0
Found existing installation: torchaudio 2.7.0
Uninstalling torchaudio-2.7.0:
  Successfully uninstalled torchaudio-2.7.0
DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install torch-directml

DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement torch-directml (from versions: none)
ERROR: No matching distribution found for torch-directml
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install torchvision torchaudio

DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached torchvision-0.22.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.7.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached torch-2.7.0-cp311-none-macosx_11_0_arm64.whl.metadata (29 kB)
Using cached torchvision-0.22.0-cp311-cp311-macosx_11_0_arm64.whl (1.9 MB)
Using cached torch-2.7.0-cp311-none-macosx_11_0_arm64.whl (68.6 MB)
Using cached torchaudio-2.7.0-cp311-cp311-macosx_11_0_arm64.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
import torch, torchaudio


In [21]:
# Are you really on your venv kernel?
import sys; print(sys.executable)

# Is your model/tensor on the DirectML device?
x = torch.randn(10, device=device)
print(x.device)

/Users/bramdewaal/anaconda3/bin/python
mps:0


-----------

In [22]:
# %pip install https://files.pythonhosted.org/packages/…/torch_directml-0.2.5.dev240914-cp311-cp311-win_amd64.whl


In [23]:
%pip install -r requirements.txt

DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.


In [24]:
def build_metadata(data_dir: str):
    """
    Analyzing and collecting all metadata from the audio files (gender, male or female)
    """
    records = []
    for root, _, files in os.walk(data_dir):
        for fname in files:
            if fname.lower().endswith(".wav"):
                path = os.path.join(root, fname)
                accent = int(fname[0])          # '1'–'5'
                gender = fname[1].lower()       # 'm' or 'f'
                records.append({"path": path, "accent": accent, "gender": gender})
    return pd.DataFrame(records)


class AccentDataset(Dataset):
    """
    PyTorch Dataset for loading, preprocessing, and feature-extracting audio.
    """
    def __init__(
        self,
        metadata_df: pd.DataFrame,
        approach: str = "raw",  # "raw" or "mel"
        max_length: int = 16000 * 5,  # 5 seconds
        sample_rate: int = 16000,
        transform: torch.nn.Module = None,
        target_transform = None
    ):
        self.df = metadata_df.reset_index(drop=True)
        self.approach = approach
        self.max_length = max_length
        self.sample_rate = sample_rate
        self.transform = transform
        self.target_transform = target_transform

        # Silence trimming (VAD)
        self.vad = torchaudio.transforms.Vad(sample_rate=sample_rate)

        # Feature transforms (for 'mel' approach)
        self.mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate,
            n_mels=64,
            n_fft=1024,
            hop_length=512
        )
        self.db_transform = torchaudio.transforms.AmplitudeToDB()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        waveform, sr = torchaudio.load(row["path"])

        # Resample if needed
        if sr != self.sample_rate:
            waveform = torchaudio.transforms.Resample(sr, self.sample_rate)(waveform)

        # Convert to mono
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # Trim leading/trailing silence
        waveform = self.vad(waveform)

        # Pad or truncate to fixed length
        length = waveform.size(1)
        if length < self.max_length:
            pad_amt = self.max_length - length
            waveform = F.pad(waveform, (0, pad_amt))
        else:
            waveform = waveform[:, :self.max_length]

        # Per-sample normalization
        waveform = (waveform - waveform.mean()) / (waveform.std() + 1e-9)

        # Optional augmentations
        if self.transform is not None:
            waveform = self.transform(waveform)

        # Feature extraction
        if self.approach == "raw":
            features = waveform  # shape: [1, max_length]
        elif self.approach == "mel":
            mel_spec = self.mel_spectrogram(waveform)
            features = self.db_transform(mel_spec)  # shape: [1, n_mels, time_steps]
        else:
            raise ValueError("approach must be 'raw' or 'mel'")

        label = row["accent"]
        if self.target_transform is not None:
            label = self.target_transform(label)

        return features, label



In [25]:
import torchaudio

In [26]:
import sys
print(sys.executable)


/Users/bramdewaal/anaconda3/bin/python


In [27]:
df = build_metadata( "Train" ) # Training dataframe based on accent & gender metadata
raw_ds = AccentDataset(df, approach="raw",  max_length=16000*5)
mel_ds = AccentDataset(df, approach="mel",  max_length=16000*5)

# Dataloaders
batch_size = 128
raw_loader = DataLoader(raw_ds, batch_size=batch_size, shuffle=True, num_workers=4)
mel_loader = DataLoader(mel_ds, batch_size=batch_size, shuffle=True, num_workers=4)

## 1.2a: Raw input signal -> analyze as 1D signal -> standardize

In [28]:
%pip install scikit-learn

DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


# Splitting into train/val (80/20) with stratification on accent, so they each appear ~ in the same proportion in train/validation set
val_fraction = 0.2
df_train, df_val = train_test_split(
    df,
    test_size=val_fraction,
    random_state=42,
    stratify=df["accent"]
)
df_train = df_train.reset_index(drop=True)
df_val   = df_val.reset_index(drop=True)



max_length = 16000 * 5  # 5 seconds
batch_size = 32
num_workers = 8

# Raw waveform datasets and loaders
train_ds = AccentDataset(
    metadata_df=df_train,
    approach="raw",       # raw 1D signal
    max_length=max_length,
)
val_ds   = AccentDataset(
    metadata_df=df_val,
    approach="raw",
    max_length=max_length,
)

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)




## Raw models

## Training models

In [30]:

try:
    import torch_directml
    device = torch_directml.device()
    print("Using DirectML on AMD GPU:", device)
except ImportError:
    import torch
    device = torch.device("cpu")
    print("DirectML unavailable—using CPU:", device)


DirectML unavailable—using CPU: cpu


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
             # ← add this

# --- 1. Define RNN and LSTM models ---


class RawCNN1D(nn.Module):
    def __init__(self, num_classes=5, p_dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=9, stride=1, padding=4), nn.ReLU(),
            nn.MaxPool1d(4),    # <-- aggressive pooling (downsample x4)
            
            nn.Conv1d(32, 64, kernel_size=9, stride=1, padding=4), nn.ReLU(),
            nn.MaxPool1d(4),    # <-- downsample again (x4)

            nn.Conv1d(64, 128, kernel_size=9, stride=1, padding=4), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),  # Final collapse to [B, 128, 1]
            
            nn.Flatten(),
            nn.Dropout(p_dropout)
        )
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.net(x)
        return self.classifier(out)


class MelCNN2D(nn.Module):
    def __init__(self, num_classes=5, p_dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Dropout(p_dropout)
        )
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        # x: [B, 1, n_mels, time]
        out = self.net(x)
        return self.classifier(out)



In [ ]:



train_ds_raw = AccentDataset(df_train, approach="raw", max_length=16000*5)
val_ds_raw   = AccentDataset(df_val, approach="raw", max_length=16000*5)

# Raw loaders
train_loader_raw = DataLoader(train_ds_raw, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader_raw   = DataLoader(val_ds_raw, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)


# Mel spectogram loaders 
train_ds_mel = AccentDataset(df_train, approach="mel", max_length=16000*5)
val_ds_mel   = AccentDataset(df_val, approach="mel", max_length=16000*5)

train_loader_mel = DataLoader(train_ds_mel, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader_mel   = DataLoader(val_ds_mel, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)




In [34]:


import torch

# Selecting device, as training was done on both AMD GPU and Apple Sillicon chip
try:
    import torch_directml
    device = torch_directml.device()
    print("Using DirectML on AMD GPU:", device)

except ImportError:
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using Apple MPS on Apple Silicon:", device)
    else:
        # Fallback to CPU
        device = torch.device("cpu")
        print("Falling back to CPU:", device)


criterion = nn.CrossEntropyLoss()


import time

def run_experiment(model_cls, train_loader, val_loader, epochs=10, **model_kwargs):
    model = model_cls(**model_kwargs).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    print(f"Training {model_cls.__name__} for {epochs} epochs...\n")

    for epoch in range(1, epochs + 1):
        epoch_start = time.time()

        # Train
        model.train()
        total_loss, correct, count = 0, 0, 0
        for batch_idx, (x, y) in enumerate(train_loader):
            print(f"Processing batch {batch_idx+1}/{len(train_loader)}")
            x, y = x.to(device, non_blocking=True), (y-1).to(device, non_blocking=True)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            count += x.size(0)

        train_loss = total_loss / count
        train_acc = correct / count

        # Validate
        model.eval()
        total_loss, correct, count = 0, 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device, non_blocking=True), (y-1).to(device, non_blocking=True)
                logits = model(x)
                loss = criterion(logits, y)
                total_loss += loss.item() * x.size(0)
                preds = logits.argmax(dim=1)
                correct += (preds == y).sum().item()
                count += x.size(0)

        val_loss = total_loss / count
        val_acc = correct / count

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        epoch_time = time.time() - epoch_start

        print(f"[{model_cls.__name__}] Epoch {epoch:02d}/{epochs} "
              f"Train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
              f"Val: loss={val_loss:.4f}, acc={val_acc:.4f} | "
              f"Time: {epoch_time:.1f}s")

    print("\nTraining finished.\n")
    return model, history



Using Apple MPS on Apple Silicon: mps


In [38]:
# Running the experiments
# Training CNN1D on raw waveforms
raw_cnn_model, raw_cnn_history = run_experiment(
    RawCNN1D,
    train_loader=train_loader_raw,
    val_loader=val_loader_raw,
    epochs=5,
    num_classes=5,
    p_dropout=0.3
)
# Five epochs for training on raw

ImportError: cannot import name 'code_framelocals_names' from 'torch._C._dynamo.eval_frame' (unknown location)

In [ ]:
print(len(train_loader_raw))   # How many batches per epoch?
print(len(train_ds_raw))       # How many total samples?

40
2532


## Analysing results

In [ ]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt

def plot_training_curves(history, model_name="Model"):
    """
    Plots train/validation loss and accuracy curves for a model.
    """
    epochs = range(1, len(history["train_loss"]) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))

    # Plot Loss
    ax1.plot(epochs, history["train_loss"], label="Train Loss")
    ax1.plot(epochs, history["val_loss"], label="Val Loss")
    ax1.set_title(f"{model_name} Loss")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.legend()

    # Plot Accuracy
    ax2.plot(epochs, history["train_acc"], label="Train Acc")
    ax2.plot(epochs, history["val_acc"], label="Val Acc")
    ax2.set_title(f"{model_name} Accuracy")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.legend()

    plt.suptitle(f"Training Curves for {model_name}", fontsize=16)
    plt.show()


In [ ]:
# plot_training_curves(raw_cnn_history, model_name="Raw CNN1D")



In [ ]:
# # Training CNN2D on Mel spectrograms
# mel_cnn_model, mel_cnn_history = run_experiment(
#     MelCNN2D,
#     train_loader=train_loader_mel,
#     val_loader=val_loader_mel,
#     num_classes=5,
#     p_dropout=0.3
# )


In [ ]:
# plot_training_curves(mel_cnn_history, model_name="Mel CNN2D")